<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/767_assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
df = pd.read_csv("./Assignment_3_Hitters.csv")
df.rename( columns={'Unnamed: 0':'hitter_name'}, inplace=True )
df

X_train = df.loc[~df.isnull().any(axis=1), df.columns != 'Salary']
Y_train = df.loc[~df.isnull().any(axis=1), 'Salary']
X_test = df.loc[df.isnull().any(axis=1), df.columns != 'Salary']


In [ ]:
Y_train

1       475.0
2       480.0
3       500.0
4        91.5
5       750.0
        ...  
317     700.0
318     875.0
319     385.0
320     960.0
321    1000.0
Name: Salary, Length: 263, dtype: float64